In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from segysak.segy import (
    get_segy_texthead,
    segy_header_scan,
    segy_header_scrape,
)
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
import pathlib

V3D_path = pathlib.Path(r"data\01_raw\VOLVE\seismic\ST10010\Stacks\ST10010ZC11_PZ_PSDM_KIRCH_FULL_T.MIG_FIN.POST_STACK.3D.JS-017536.segy")
seisnc = xr.open_dataset(
    V3D_path,
    # engine="netcdf4",
    dim_byte_fields={"iline": 189, "xline": 193},
    extra_byte_fields={"cdp_x": 181, "cdp_y": 185},
)

seisnc.segysak.scale_coords()

print(seisnc.seis.humanbytes)
print(seisnc.chunks)

In [ ]:
get_segy_texthead(V3D_path)

In [ ]:
import pandas as pd

well_survey = pd.read_csv(r"data\01_raw\VOLVE\VOLVE_15-9_Surveys.csv")

well_survey.head()

In [ ]:
from quick_pp.geophysics.seismic import extract_seismic_along_well

# Extract seismic along well trajectory
# First prepare well coordinates dataframe
well_name = '15-9-19-BT2'
well_coords = well_survey[well_survey['WELL_NAME'] == well_name][
    ['MD (m RKB)', 'Inc (deg)', 'Azim (deg)', 'UTM E/W (m)', 'UTM N/S (m)', 'TVD (m RKB)']
]
well_coords.columns = ['md', 'incl', 'azim', 'x', 'y', 'z']

In [ ]:
well_coords.tail()

In [ ]:
from quick_pp.rock_physics.geophysics import *

fig = plot_well_trajectory(seisnc, well_coords)

In [ ]:
fig = plot_seismic_along_well(seisnc, well_coords)

In [ ]:
import quick_pp.las_handler as las

with open(rF'data\01_raw\VOLVE\{well_name}.las', 'rb') as f:
    df, header = las.read_las_files([f])

In [ ]:
clean_df = df.copy()
clean_df['GR'] = clean_df.GR.clip(0, 200)

well_seismic = extract_seismic_along_well(seisnc, well_coords)
well_trace_df = well_seismic.to_dataframe().reset_index(names='DEPTH')
well_trace_df['TRACE'] = well_trace_df['data']
ref = pd.merge_asof(clean_df, well_trace_df, on='DEPTH', tolerance=0.1, direction='nearest')

layers = auto_identify_layers_from_seismic_trace(ref, 'zero_crossings')
# ref = calculate_reflectivity_from_layers(ref)
ref = calculate_reflectivity_each_step(ref)
ref.plot(x='DEPTH', y='REFLECTIVITY')

In [ ]:
ref.REFLECTIVITY.describe()

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(15, 5), sharex=True)
axs[0].plot(ref.DEPTH, ref.TRACE, label='TRACE')
axs[1].plot(ref.DEPTH, ref.LAYERS)
axs[2].plot(ref.DEPTH, ref.AI, label='AI')
# axs[2].plot(ref.AVG_AI, label='AVG_AI')
axs[2].legend()
axs[3].plot(ref.DEPTH, ref.AI.rolling(100).mean().diff().clip(lower=-5000, upper=5000), label='AI Diff')
axs[4].plot(ref.DEPTH, ref.REFLECTIVITY, label='Reflectivity')
axs[4].legend()

In [ ]:
ref.tail()

In [ ]:
well_trace_df.tail()

In [ ]:
well_trace_df.TRACE.plot()

In [ ]:
W = optimize_wavelet(ref.TRACE, ref.REFLECTIVITY.fillna(0))

plt.plot(W[0], label='Wavelet')
plt.legend()

fig, axs = plt.subplots(2, 1, figsize=(15, 5), sharex=True)
axs[0].plot(ref.TRACE, label='Seismic')
axs[0].plot(W[2], label='Synthetic Seismic')
axs[0].legend()
axs[1].plot(ref.REFLECTIVITY, label='Reflectivity')
axs[1].legend()

In [ ]:
from scipy.signal import convolve, hilbert

def ricker_wavelet(length, f0=30, amplitude=1.0, phase=0.0, asymmetry=3):
    """Generate a Ricker wavelet with specified parameters.
    
    Args:
        length (int): Length of wavelet in samples
        f0 (float): Peak frequency in Hz
        amplitude (float): Maximum amplitude of wavelet
        phase (float): Phase rotation in degrees
        asymmetry (float): Asymmetry factor (-1 to 1) to skew the wavelet
    
    Returns:
        numpy.ndarray: Ricker wavelet
    """
    t = np.linspace(-length/2, length/2, length) / 1000  # Convert to seconds
    pi2 = np.pi * np.pi
    
    # Generate zero-phase Ricker wavelet
    w = (1 - 2*pi2*f0**2*t**2) * np.exp(-pi2*f0**2*t**2)
    w = w / np.max(np.abs(w))  # Normalize
    
    # Add asymmetry by warping the time axis
    if asymmetry != 0:
        t_warped = t + asymmetry * t**2
        w = np.interp(t, t_warped, w)
    
    # Apply phase rotation
    phase_rad = np.deg2rad(phase)
    w = w * np.cos(phase_rad) + np.imag(hilbert(w)) * np.sin(phase_rad)
    
    # Apply amplitude scaling
    w = w * amplitude
    
    # Apply tapering to reduce edge effects
    taper = np.hanning(len(w))

    return w * taper

rw = ricker_wavelet(
    length=31,
    f0=.25,
    amplitude=1,
    phase=0
)

plt.plot(rw, label='Ricker Wavelet')
plt.legend()

fig, axs = plt.subplots(2, 1, figsize=(15, 5), sharex=True)
axs[0].plot(ref.TRACE, label='Seismic')
# axs[0].plot(W[2], label='Synthetic Seismic')
axs[0].plot(convolve(ref.REFLECTIVITY, rw, mode='same'), label='Synthetic Seismic Convolved')
axs[0].legend()
axs[1].plot(ref.REFLECTIVITY, label='Reflectivity')
axs[1].legend()